In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib import colors
import io

model = joblib.load('best_model.pkl')
preprocessor = joblib.load('preprocessor.pkl')
st.set_page_config(
    page_title = "Credit Card Approval Detection",
    layout = "centered"
)

st.title("💳💳 Credit Card Approval")

2026-01-28 10:03:47.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:03:47.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:03:47.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:03:47.525 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
with st.form("credit_form"):
  st.subheader("Applicant Information👤")

  gender = st.selectbox("Gender♂️♀️", ["M", "F"])
  own_car = st.selectbox("Owns a Car?🚗", ["Y", "N"])
  own_realty = st.selectbox("Owns Property?🏠", ["Y", "N"])
  children = st.number_input("Number of Children👩‍👧‍👦", 0, 10, 0)

  income = st.number_input("Annual Income💵", 0, 1000000, 50000)
  income_type = st.selectbox(
        "Income Type",
        ["Working", "Commercial associate", "Pensioner", "State servant", "Student"]
    )

  education = st.selectbox(
        "Education Level",
        ["Secondary / secondary special", "Higher education", "Incomplete higher"]
    )

  family_status = st.selectbox(
        "Family Status",
        ["Married", "Single / not married", "Civil marriage", "Separated", "Widow"]
    )

  housing = st.selectbox(
        "Housing Type",
        ["House / apartment", "Rented apartment", "With parents"]
    )

  occupation = st.selectbox(
        "Occupation",
        [
            "Laborers", "Core staff", "Managers", "Drivers",
            "Sales staff", "Accountants", "Unknown"
        ]
    )

  days_birth = st.number_input("Age (years)", 18, 100, 30)
  days_employed = st.number_input("Years Employed", 0, 50, 5)

  submit = st.form_submit_button("Submit")

2026-01-28 10:21:38.649 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:21:38.650 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:21:38.651 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:21:38.653 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:21:38.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:21:38.655 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:21:38.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-28 10:21:38.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
def generate_pdf(data, probability, decision):
    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=A4)
    width, height = A4

    c.setFont("Helvetica-Bold", 18)
    c.drawString(50, height - 50, "Credit Card Approval Report📂")

    c.setFont("Helvetica", 11)
    c.drawString(50, height - 90, f"Final Decision: {decision}")
    c.drawString(50, height - 110, f"Approval Probability: {probability:.2%}")

    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, height - 150, "🧾Applicant Details:")

    y = height - 180
    c.setFont("Helvetica", 10)

    for key, value in data.items():
        c.drawString(60, y, f"{key}: {value}")
        y -= 15
        if y < 50:
            c.showPage()
            c.setFont("Helvetica", 10)
            y = height - 50

    c.showPage()
    c.save()
    buffer.seek(0)

    return buffer


In [ ]:
if submit:
  data = {
        "CODE_GENDER": gender,
        "FLAG_OWN_CAR": own_car,
        "FLAG_OWN_REALTY": own_realty,
        "CNT_CHILDREN": children,
        "AMT_INCOME_TOTAL": income,
        "NAME_INCOME_TYPE": income_type,
        "NAME_EDUCATION_TYPE": education,
        "NAME_FAMILY_STATUS": family_status,
        "NAME_HOUSING_TYPE": housing,
        "OCCUPATION_TYPE": occupation,
        "DAYS_BIRTH": days_birth * 365,
        "DAYS_EMPLOYED": days_employed * 365,
        "HAS_OCCUPATION": 0 if occupation == "Unknown" else 1
  }

  model_input = {
        "CODE_GENDER": gender,
        "FLAG_OWN_CAR": own_car,
        "FLAG_OWN_REALTY": own_realty,
        "CNT_CHILDREN": children,
        "AMT_INCOME_TOTAL": income,
        "NAME_INCOME_TYPE": income_type,
        "NAME_EDUCATION_TYPE": education,
        "NAME_FAMILY_STATUS": family_status,
        "NAME_HOUSING_TYPE": housing,
        "OCCUPATION_TYPE": occupation,
        "DAYS_BIRTH": days_birth * 365,
        "DAYS_EMPLOYED": days_employed * 365,
        "HAS_OCCUPATION": 0 if occupation == "Unknown" else 1
    }

  input_df = pd.DataFrame([model_input])
  x_processed = preprocessor.transform(input_df)
  prob = model.predict_prob(x_processed)[0][1]

  threshold = 0.6
  decision = "Approved✅" if prob >= threshold else "Denied⛔"

  st.subheader("📊 Prediction Result")
  st.markdown(f"**{decision}**")
  st.write(f"Approval Probability: {prob:.2%}")

  pdf = generate_pdf(data, prob, decision)

  st.download_button(
      label="Download PDF",
      data=pdf,
      file_name="credit_card_approval_report.pdf",
      mime="application/pdf"
  )
